In [1]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
import torch
import os
import numpy as np
import faiss
import pandas as pd
import matplotlib.pyplot as plt
import time
import torchaudio
import gc
import math
bundle= torchaudio.pipelines.HUBERT_BASE
model= bundle.get_model()
directory= "/Users/ariel/Desktop/Hubert-PING/cats_dogs/"


/Users/ariel/anaconda3/envs/faiss/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np


# Définition de la fonction exponentielle négative
def exp_negative(x):
    return math.exp(-x)

#j'ai du ajouter un traitement pour les vecteurs 1D car le vecteur de requete contient une seul vecteur et ne peut etre en 2D car comme tu le sais notre processus de traitement des audios nous donne une longue liste contenant tous les caractéritiques dont on a besoin, voila pourquoi le vecteur de requeue est en 1 dimension j'ai donc améliorer la logique de mon code 
def normalization(embeddings):
    # Convertir en un tableau NumPy si ce n'est pas déjà le cas
    #embeddings = np.array(embeddings)

    # Vérifie si embeddings est un vecteur seul (1D) ou une matrice (2D)
    if embeddings.ndim == 1:
        # Normaliser un vecteur seul
        norm = np.linalg.norm(embeddings)
        if norm == 0:
            return embeddings
        return embeddings / norm
    else:
        # Normaliser chaque ligne d'une matrice
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        return embeddings / norms
    
def get_audio_embedding(audio_path):
    waveform1, sample_rate1=torchaudio.load(audio_path)
    waveform1=torchaudio.functional.resample(waveform1, sample_rate1, bundle.sample_rate)
    with torch.inference_mode():
            emission1, _ = model(waveform1)

    # Aplatir les deux premières dimensions et garder la troisième
    flattened_features1 = emission1.view(-1, emission1.size(2))
    mean_features1 = flattened_features1.mean(dim=0)
    mean1_array=mean_features1.cpu().numpy().astype(np.float32) 
    mean1_normal=normalization(mean1_array)
    mean1_normal_2d = mean1_normal[np.newaxis, :]
    return mean1_normal_2d


def searchinIndex(index, normal_embedding):
    D,I= index.search(normal_embedding, index.ntotal)
    r=pd.DataFrame({'distance':D[0],'index':I[0]})
    return r

In [3]:
index = faiss.read_index("/Users/ariel/Downloads/animals.index")

In [4]:
chemin_noms_animaux = '/Users/ariel/Downloads/noms_animaux.txt'

# Lire le contenu du fichier et le convertir en liste
with open(chemin_noms_animaux, 'r') as fichier:
    # Utiliser une compréhension de liste pour traiter chaque ligne
    names = [line.strip().strip("'").strip(",").strip() for line in fichier.readlines()]



In [5]:
query_audio=get_audio_embedding("/Users/ariel/Desktop/Dev/Hubert-PING/tests/birds_216.wav")

In [6]:
index.ntotal

4433

In [7]:
len(names)

4433

In [8]:

i=searchinIndex(index,query_audio)

In [9]:
print(i)

      distance  index
0     0.000000   4432
1     0.312191   2387
2     0.323126   4390
3     0.330927   3713
4     0.332973   2883
...        ...    ...
4428  1.484970   2934
4429  1.492944   3514
4430  1.493668   1376
4431  1.553486   3442
4432  1.568810   1344

[4433 rows x 2 columns]


In [10]:
i['percentage'] = i['distance'].apply(exp_negative)*100

In [11]:
i

,distance,index,percentage
0,0.000000,4432,100.000000
1,0.312191,2387,73.184186
2,0.323126,4390,72.388243
3,0.330927,3713,71.825785
4,0.332973,2883,71.678933
...,...,...,...
4428,1.484970,2934,22.650911
4429,1.492944,3514,22.471024
4430,1.493668,1376,22.454754
4431,1.553486,3442,21.150927


In [12]:
def get_name_from_index(index):
    return names[index]

i['names'] = i['index'].apply(get_name_from_index)

In [13]:
i

,distance,index,percentage,names
0,0.000000,4432,100.000000,birds_216.wav'
1,0.312191,2387,73.184186,cat_70_stretching.wav''''
2,0.323126,4390,72.388243,cat_158.wav''''
3,0.330927,3713,71.825785,birds_428.wav''''
4,0.332973,2883,71.678933,cat_601_stretching.wav''''
...,...,...,...,...
4428,1.484970,2934,22.650911,dogs_591_stretching.wav''''
4429,1.492944,3514,22.471024,dogs_591.wav''''
4430,1.493668,1376,22.454754,dogs_333_stretching.wav''''
4431,1.553486,3442,21.150927,dogs_521.wav''''


In [14]:
def name_normalisation(name):
    if 'dog' in name:
        return "Chien"
    elif 'cat' in name:
        return "Chat"
    elif 'bird' in name:
        return "Oiseau"
    else:
        return "Animal non reconnu"



In [15]:
i['names_normalized'] = i['names'].apply(name_normalisation)

In [16]:
i

,distance,index,percentage,names,names_normalized
0,0.000000,4432,100.000000,birds_216.wav',Oiseau
1,0.312191,2387,73.184186,cat_70_stretching.wav'''',Chat
2,0.323126,4390,72.388243,cat_158.wav'''',Chat
3,0.330927,3713,71.825785,birds_428.wav'''',Oiseau
4,0.332973,2883,71.678933,cat_601_stretching.wav'''',Chat
...,...,...,...,...,...
4428,1.484970,2934,22.650911,dogs_591_stretching.wav'''',Chien
4429,1.492944,3514,22.471024,dogs_591.wav'''',Chien
4430,1.493668,1376,22.454754,dogs_333_stretching.wav'''',Chien
4431,1.553486,3442,21.150927,dogs_521.wav'''',Chien


In [17]:
def bayes_theorem(df, n_top_vectors=50):
    # Limite le DataFrame aux n_top_vectors premiers vecteurs
    df_limited = df.head(n_top_vectors)
    # Récupère les catégories uniques et initialise le dictionnaire des probabilités a posteriori
    categories = df_limited['names_normalized'].unique()
    probas_a_posteriori = {categorie: 0 for categorie in categories}
    # Calcul des probabilités a priori uniformes
    probas_a_priori = 1/3
    # Somme des similarités pour chaque catégorie limitée aux 50 premiers vecteurs
    for categorie in categories:
        somme_similarites = df_limited[df_limited['names_normalized'] == categorie]['percentage'].sum()
        probas_a_posteriori[categorie] = somme_similarites * probas_a_priori
    # Normalisation des probabilités a posteriori
    total_proba = sum(probas_a_posteriori.values())
    probas_a_posteriori_normalisees = {categorie: (proba / total_proba) for categorie, proba in probas_a_posteriori.items()}
    return probas_a_posteriori_normalisees
 
# Supposons que 'df' est votre DataFrame
# df = pd.read_csv('chemin_vers_votre_fichier.csv')  # Exemple de chargement de données
 
# Utilisez la fonction sur les 50 premiers vecteurs de votre DataFrame
resultat = bayes_theorem(i, 50)
print(resultat)

{'Oiseau': 0.18692323034689698, 'Chat': 0.7537555200075591, 'Chien': 0.059321249645543986}


In [18]:
len(names)

4433

In [19]:
index.ntotal

4433

In [20]:
len(names)

4433

In [21]:
index.ntotal

4433

In [22]:
import torchaudio